# Dataset Diabetes

In [1]:
# fazer versão que utiliza impureza de gini

## Importações

In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import math
import statistics

## Leitura de dados

In [3]:
treinamento = pd.read_csv('dataset/diabetes_train.csv', sep=',')

In [4]:
teste = pd.read_csv('dataset/diabetes_test.csv', sep=',')

In [5]:
x_treinamento, x_teste = treinamento.iloc[:, :-1], teste.iloc[:, :-1]

In [6]:
y_treinamento, y_teste = treinamento.iloc[:, -1], teste.iloc[:, -1]

In [7]:
x_teste = x_teste.to_dict('records')

## Decision Tree

In [8]:
class No:
    def __init__(self, tipo, feature = None, valor = None):
        self.tipo = tipo
        self.feature = feature
        self.valor = valor
        self.conexoes = {}
    
    def inicializar_conexoes(self):
        for i in self.feature.atributos:
            self.conexoes[i] = None


In [9]:
class Componente:
    def __init__(self, nome, atributos, tipo, prob_priori = None, prob_condicional = None, entropia = None, ganho = 0):
        self.nome = nome
        self.atributos = atributos
        self.tipo = tipo
        self.prob_priori = prob_priori
        self.prob_condicional = prob_condicional
        if tipo != 'target': self.entropia = {}
        else: self.entropia = entropia
        self.ganho = ganho
    
    def calcula_prob_priori(self, valores_feature):
        buffer = dict(Counter(valores_feature))
        soma = sum(buffer.values())
        for i in buffer:
            buffer[i] = buffer[i]/soma
        self.prob_priori = buffer
    
    def calcula_prob_condicional(self, atributos_target, valores_target, valores_feature):
        buffer = {i: {} for i in self.atributos}

        for i in buffer:
            buffer[i] = {j: 0 for j in atributos_target}

        for i, j in zip(valores_feature, valores_target):
            buffer[i][j] += 1

        for i in buffer:
            soma = sum(buffer[i].values())
            for j in buffer[i]:
                buffer[i][j] = buffer[i][j]/soma

        self.prob_condicional = buffer
        

In [10]:
class ArvoreDecisao:
    def __init__(self):
        self.raiz = None
        self.x = None
        self.y = None

    def set_raiz(self, raiz):
        self.raiz = raiz

    def calcula_entropia(self, s):
        entropia = 0
        for i in s:
            if i != 0:
                entropia -= i * math.log(i, 2)
        
        return entropia

    def calcula_ganho(self, feature, target):
        for i in feature.atributos:
            feature.ganho += feature.entropia[i] * feature.prob_priori[i]
        
        feature.ganho = target.entropia - feature.ganho

    def calcula_maior_ganho(self, features):
        ganhos = [features[i].ganho for i in features]
        
        for i in features:
            if features[i].ganho == max(ganhos):
                return features[i]
        
        return None

    def calcula_nova_base_dados(self, no, conexao, target, labels_features, indices_base_atual):
        indices = []

        for i in indices_base_atual:
            if self.x.iloc[i][no.nome] == conexao:
                indices.append(i)
        
        labels_x = [i for i in labels_features if i != no.nome]
        meta_dados_x = [np.array(indices), labels_x]
        meta_dados_y = [np.array(indices), target]
        
        return meta_dados_x, meta_dados_y

    def calcula_arvore_decisao(self, meta_dados_x, meta_dados_y):
        labels_features, label_target = meta_dados_x[1], meta_dados_y[1]
        
        x_treinamento, y_treinamento = self.x.iloc[meta_dados_x[0]][meta_dados_x[1]].values, np.array([self.y.values[i] for i in meta_dados_y[0]])

        target = Componente(label_target, list(Counter(y_treinamento).keys()), 'target')
        target.calcula_prob_priori(y_treinamento)
        target.entropia = self.calcula_entropia(list(target.prob_priori.values()))

        features = {}

        for i, j in enumerate(labels_features):
            features[j] = Componente(j, list(Counter(x_treinamento.T[i]).keys()), 'feature')
            features[j].calcula_prob_condicional(list(Counter(y_treinamento).keys()), y_treinamento, x_treinamento.T[i])
            features[j].calcula_prob_priori(x_treinamento.T[i])

            for i in features[j].atributos:
                features[j].entropia[i] = self.calcula_entropia(list(features[j].prob_condicional[i].values()))

            self.calcula_ganho(features[j], target)

        feature = self.calcula_maior_ganho(features)
        
        if feature.ganho != 0:
            no = No('no', feature)
            no.inicializar_conexoes()
            indices_base_atual = meta_dados_x[0]

            for i in no.conexoes:
                meta_dados_x, meta_dados_y = self.calcula_nova_base_dados(no.feature, i, target, labels_features, indices_base_atual)
                no.conexoes[i] = self.calcula_arvore_decisao(meta_dados_x, meta_dados_y)
            return no
        
        elif len(list(Counter(y_treinamento).keys())) == 1:
            return No('folha', None, list(Counter(y_treinamento).keys())[0])
        
        return No('folha')
    
    def calcula_folha_maior_frequencia(self, no):
        lista = []
        for i in no.conexoes:
            j = no.conexoes[i]
            if(j.tipo == 'folha' and j.valor != None):
                lista.append(j.valor)

        return statistics.mode(lista)
    
    def verifica_folhas_none(self, no):
        lista = []
        for i in no.conexoes:
            j = no.conexoes[i]
            if(j.tipo == 'folha'):
                if(j.valor == None):
                    j.valor = self.calcula_folha_maior_frequencia(no)
            else:
                lista.append(j)
        
        for i in lista:
            self.verifica_folhas_none(i)
        
    def exibir_arvore_decisao(self, no):
        lista = []

        print("==========================")
        print(f"Nó atual: {no.feature.nome}")

        for i in no.conexoes:
            j = no.conexoes[i]
            print(f"Atributo: {i} -> {j.tipo}")
            if(j.tipo != 'folha'):
                print(f"\t -> {j.feature.nome}")
                lista.append(j)
            else:
                print(f"\tResultado: {j.valor}")

        print("==========================\n\n")

        for i in lista:
            self.exibir_arvore_decisao(i)
    
    def fit(self, x_treinamento, y_treinamento):
        if not isinstance(x_treinamento, pd.DataFrame) or not isinstance(y_treinamento, pd.Series):
            print("Atenção: os dados ser um dataframe e uma série, respectivamente!\n")
            return
        
        self.x, self.y = x_treinamento, y_treinamento

        meta_dados_x = [np.array(range(self.x.index.stop)), list(self.x.columns)]
        meta_dados_y = [np.array(range(self.y.index.stop)), self.y.name]

        self.set_raiz(self.calcula_arvore_decisao(meta_dados_x, meta_dados_y))
        self.verifica_folhas_none(self.raiz)

    def arvore_decisao(self, instancia, no):
        if no.tipo != 'folha':
            for i in no.conexoes:
                if i == instancia[no.feature.nome]:
                    resultado = self.arvore_decisao(instancia, no.conexoes[i])
        else: 
            return no.valor
        
        return resultado

    def predict(self, x_teste):
        resultados = []

        for i in x_teste:
            resultados.append(self.arvore_decisao(i, self.raiz))

        return resultados
    
    def score(self, y_resultado, y_teste):
        matriz_confusao = np.zeros((2, 2))

        for i, k in zip(y_teste, y_resultado):
            matriz_confusao[i][k] += 1
        
        acuracia = np.sum(np.diagonal(matriz_confusao))/np.sum(matriz_confusao) * 100
        precisao = (matriz_confusao[1][1]/(np.sum(matriz_confusao[:, 1]))) * 100
        revocacao = (matriz_confusao[1][1] / (np.sum(matriz_confusao[1][:]))) * 100
        f1_score = (2/((1/precisao) + 1/(revocacao)))

        print("=" * 10 + " Medições de Desempenho " + "=" * 10)
        print("Matriz de Confusão")
        print(f"{pd.DataFrame(matriz_confusao, columns=['0', '1'])}")
        print(f"\nAcurácia: {acuracia:.2f}%")
        print(f"Precisão: {precisao:.2f}%")
        print(f"Revocação: {revocacao:.2f}%")
        print(f"F1-Score: {f1_score:.2f}%")
        print("=" * 44)


In [11]:
arvore = ArvoreDecisao()
arvore.fit(x_treinamento, y_treinamento)
arvore.exibir_arvore_decisao(arvore.raiz)
resultados = arvore.predict(x_teste)

Nó atual: polyuria
Atributo: 1 -> no
	 -> polydipsia
Atributo: 0 -> no
	 -> polydipsia


Nó atual: polydipsia
Atributo: 1 -> folha
	Resultado: 1
Atributo: 0 -> no
	 -> delayed_healing


Nó atual: delayed_healing
Atributo: 0 -> folha
	Resultado: 1
Atributo: 1 -> no
	 -> obesity


Nó atual: obesity
Atributo: 0 -> no
	 -> weakness
Atributo: 1 -> folha
	Resultado: 0


Nó atual: weakness
Atributo: 1 -> folha
	Resultado: 1
Atributo: 0 -> no
	 -> age


Nó atual: age
Atributo: 60<_<=80 -> folha
	Resultado: 0
Atributo: 40<_<=60 -> folha
	Resultado: 1
Atributo: 20<_<=40 -> folha
	Resultado: 1


Nó atual: polydipsia
Atributo: 0 -> no
	 -> gender
Atributo: 1 -> no
	 -> muscle_stiffness


Nó atual: gender
Atributo: Male -> no
	 -> irritability
Atributo: Female -> no
	 -> alopecia


Nó atual: irritability
Atributo: 0 -> no
	 -> delayed_healing
Atributo: 1 -> no
	 -> genital_thrush


Nó atual: delayed_healing
Atributo: 0 -> folha
	Resultado: 0
Atributo: 1 -> no
	 -> age


Nó atual: age
Atributo: 40<_

In [12]:
arvore.score(resultados, y_teste)

========== Medições de Desempenho ==========
Matriz de Confusão
      0      1
0  79.0    2.0
1  10.0  117.0

Acurácia: 94.23%
Precisão: 98.32%
Revocação: 92.13%
F1-Score: 95.12%
